In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

population_00 = pd.read_csv("../Raochuan/nyc_tract_2000.csv")
population_00.drop('Total Population',1,inplace = True)
population_00.drop('Median Age',1,inplace = True)
population_00.columns.values[1] = '2010 Census Tract'
population_00.head()
population_01 = pd.read_csv("../Haichao/2001_predict_backward.csv")
population_02 = pd.read_csv("../Haichao/2002_predict_backward.csv")
population_03 = pd.read_csv("../Haichao/2003_predict_backward.csv")
population_04 = pd.read_csv("../Haichao/2004_predict_backward.csv")
population_05 = pd.read_csv("../Haichao/2005_predict_backward.csv")
population_06 = pd.read_csv("../Haichao/2006_predict_backward.csv")
population_07 = pd.read_csv("../Haichao/2007_predict_backward.csv")
population_08 = pd.read_csv("../Haichao/2008_predict_backward.csv")
population_09 = pd.read_csv("../Haichao/2009_predict_backward.csv",names = population_08.columns.values[2:])
population_09.insert(0,'Borough',population_08['Borough'])
population_09.insert(1,'2010 Census Tract',population_08['2010 Census Tract'])
population_10 = pd.read_csv("nyc_tract_2010.csv")
population_10.drop('Total Population',1,inplace = True)
population_10.drop('Median Age',1,inplace = True)

In [2]:
# The 2000 data is special, so we handle it indepedently
raw_data = pd.read_csv("../Raochuan/Kth_09_10.csv")
ct_index = raw_data['index']
population_00 = population_00.ix[population_00['Borough'] == 'Brooklyn']
population_00
ct_index[ct_index % 1 != 0]=ct_index*100
ct_index = pd.DataFrame(data = ct_index)

data_2000 = pd.merge(ct_index,population_00,left_on='index',right_on='2010 Census Tract',how='left')
data_2000 = data_2000[['index', 'Under 5 Years', '5-9      Years', '10-14 Years']]
for i in range(len(data_2000['index'])):
    if (data_2000.iloc[i]['index'] > 1144):
        data_2000.set_value(data_2000.index.values[i],'index', data_2000.iloc[i]['index']*0.01)
data_2000.columns = ['2010 Census Tract', 'Under 5 Years', '5-9      Years', '10-14 Years']
data_2000.fillna(0, inplace = True)
data_2000.dtypes

2010 Census Tract    float64
Under 5 Years         object
5-9      Years        object
10-14 Years           object
dtype: object

In [3]:
def filter_brooklyn_20(yearly_data):
    yearly_data = yearly_data.ix[yearly_data['Borough'] == 'Brooklyn']
    yearly_data['2010 Census Tract'] = [x / 100 for x in yearly_data['2010 Census Tract']]
    raw_data = pd.read_csv("../Raochuan/Kth_09_10.csv")
    ct_index = raw_data['index']
    yearly_data = yearly_data.ix[yearly_data['2010 Census Tract'].isin(ct_index)]
    yearly_data = yearly_data[['2010 Census Tract', 'Under 5 Years', '5-9      Years', '10-14 Years']]
    return yearly_data

In [4]:
data_2000 = data_2000.convert_objects(convert_numeric=True)
data_2001 = filter_brooklyn_20(population_01)
data_2002 = filter_brooklyn_20(population_02)
data_2003 = filter_brooklyn_20(population_03)
data_2004 = filter_brooklyn_20(population_04)
data_2005 = filter_brooklyn_20(population_05)
data_2006 = filter_brooklyn_20(population_06)
data_2007 = filter_brooklyn_20(population_07)
data_2008 = filter_brooklyn_20(population_08)
data_2009 = filter_brooklyn_20(population_09)
data_2010 = filter_brooklyn_20(population_10)

In [5]:
def diff(former_data, latter_data,year):
    diff = pd.DataFrame(columns=['2010 Census Tract', year+'_0-4', year+'_5-9', year+'_10-14'])
    diff['2010 Census Tract'] = former_data['2010 Census Tract']
    diff[year+'_0-4'] = [a - b for a, b in zip(latter_data['Under 5 Years'], former_data['Under 5 Years'])]
    diff[year+'_5-9'] = [a - b for a, b in zip(latter_data['5-9      Years'], former_data['5-9      Years'])]
    diff[year+'_10-14'] = [a - b for a, b in zip(latter_data['10-14 Years'], former_data['10-14 Years'])]
    return diff

In [6]:
diff20002001 = diff(data_2000, data_2001, "20002001")
diff20012002 = diff(data_2001, data_2002, "20012002")
diff20022003 = diff(data_2002, data_2003, "20022003")
diff20032004 = diff(data_2003, data_2004, "20032004")
diff20042005 = diff(data_2004, data_2005, "20042005")
diff20052006 = diff(data_2005, data_2006, "20052006")
diff20062007 = diff(data_2006, data_2007, "20062007")
diff20072008 = diff(data_2007, data_2008, "20072008")
diff20082009 = diff(data_2008, data_2009, "20082009")
diff20092010 = diff(data_2009, data_2010, "20092010")

diff20002001.head()

,2010 Census Tract,20002001_0-4,20002001_5-9,20002001_10-14
0,18.0,-4,-1,1
1,20.0,4,24,27
2,22.0,20,19,-22
3,30.0,-16,-9,10
4,34.0,20,50,30


In [16]:
data = raw_data[['index']]
for i in range(len(data['index'])):
    if (data.iloc[i]['index'] > 1144):
        data.set_value(data.index.values[i],'index', data.iloc[i]['index']*0.01)
data.columns=['2010 Census Tract']
data['Wht%'] = raw_data['White'] / raw_data['total']
data['Blc%'] = raw_data.iloc[:,6] / raw_data['total']
data['Ind%'] = raw_data.iloc[:,7] / raw_data['total']
data['Asn%'] = raw_data.iloc[:,8] / raw_data['total']
data['Mix%'] = raw_data.iloc[:,11] / raw_data['total']
data['His%'] = raw_data.iloc[:,12] / raw_data['total']


In [8]:
def insert_data(integrated_data, insert_data):
    integrated_data[insert_data.columns[1]] = insert_data.iloc[:,1]
    integrated_data[insert_data.columns[2]] = insert_data.iloc[:,2]
    integrated_data[insert_data.columns[3]] = insert_data.iloc[:,3]
    print(integrated_data)
    return integrated_data

In [17]:
new = pd.merge(data,diff20002001,how='left',on='2010 Census Tract')
new = pd.merge(new,diff20012002,how='left',on='2010 Census Tract')
new = pd.merge(new,diff20022003,how='left',on='2010 Census Tract')
new = pd.merge(new,diff20032004,how='left',on='2010 Census Tract')
new = pd.merge(new,diff20042005,how='left',on='2010 Census Tract')
new = pd.merge(new,diff20052006,how='left',on='2010 Census Tract')
new = pd.merge(new,diff20062007,how='left',on='2010 Census Tract')
new = pd.merge(new,diff20072008,how='left',on='2010 Census Tract')
new = pd.merge(new,diff20082009,how='left',on='2010 Census Tract')
new = pd.merge(new,diff20092010,how='left',on='2010 Census Tract')
new.to_csv("CTdata.csv")

In [19]:
new.shape

(157, 37)